# Market Basket Analysis and Association Rule
---
Market Basket Analysis is one of the key techniques used by large retailers to uncover associations between items. It works by looking for combinations of items that occur together frequently in transactions. To put it another way, it allows retailers to identify relationships between the items that people buy.

![](https://editor.analyticsvidhya.com/uploads/13952Market-basket-analysis.png)

## Frequent Itemset Mining (FIM)
---
Frequent Itemset Mining is a method that comes under the market basket analysis. Frequent Itemset mining aims to find the regularities in the transactions performed by the consumers. In terms of the supermarket, we can say regularities in the shopping behaviour of customers. 

Basically, frequent itemset mining is a procedure that helps in finding the sets of products that are frequently bought together. Found frequent itemsets can be applied on the procedure of recommendation system, fraud detection or using them we can improve the arrangement of the products in the shelves.

#### [Metric] Support
Support is a measure that indicates the frequent appearance of a variable set or itemset in a database. Let X be the itemset and T a set of transactions in  then the support of X with respect to T can be measured as  
![](https://149695847.v2.pressablecdn.com/wp-content/uploads/2021/09/image-126.png)  
Basically, the above measure tells the proportion of T transactions in the database which contains the item set X. From above the given table support for itemset  {product 1, product 2} will be ⅖ or 0.4. Because the itemset has appeared only in 2 transactions and the total count of transactions is 5.

### Apriori
---
In the process of Frequent Itemset Mining, the Apriori algorithm first considers every single item as an itemset and counts the support from their frequency in the database, and then captures those who have support equal to or more than the minimum support threshold. In this process extraction of each frequent itemset requires the scanning of the entire database by the algorithm until no more itemsets are left with more than the minimum threshold support.  
![](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1530898581/Image_5_qcih6b.png)
![](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1530898581/Image_6_ee2pss.png)

In [ ]:
!pip install mlxtend -q

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-8bf9403a-0535-46fc-bc08-c4650b888a70/bin/python -m pip install --upgrade pip' command.


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from pyspark.sql.functions import *
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.types import *

import pandas as pd
import numpy as np

For implementation, I am making a dataset of 11 products and using the mlxtend library for making a Frequent dataset using the apriori algorithm.

In [ ]:
txn_id = ['txn1', 'txn2', 'txn3', 'txn4', 'txn5']
dataset = [['product7', 'product9', 'product8', 'product6', 'product4', 'product11'],
           ['product3', 'product9', 'product8', 'product6', 'product4', 'product11'],
           ['product7', 'product1', 'product6', 'product4'],
           ['product7', 'product10', 'product2', 'product6', 'product11'],
           ['product2', 'product9', 'product6', 'product5', 'product4']]
df = pd.DataFrame({'txn_id': txn_id, 'items': dataset})
df

,txn_id,items
0,txn1,"[product7, product9, product8, product6, produ..."
1,txn2,"[product3, product9, product8, product6, produ..."
2,txn3,"[product7, product1, product6, product4]"
3,txn4,"[product7, product10, product2, product6, prod..."
4,txn5,"[product2, product9, product6, product5, produ..."


In [ ]:
con = TransactionEncoder()
con_arr = con.fit(dataset).transform(dataset)
df = pd.DataFrame(con_arr, columns = con.columns_)
df

,product1,product10,product11,product2,product3,product4,product5,product6,product7,product8,product9
0,False,False,True,False,False,True,False,True,True,True,True
1,False,False,True,False,True,True,False,True,False,True,True
2,True,False,False,False,False,True,False,True,True,False,False
3,False,True,True,True,False,False,False,True,True,False,False
4,False,False,False,True,False,True,True,True,False,False,True


Next, I will be making itemsets with at least 60% support.

In [ ]:
apriori(df, min_support=0.6, use_colnames=True).sort_values('support', ascending=False)

,support,itemsets
2,1.0,(product6)
1,0.8,(product4)
6,0.8,"(product6, product4)"
0,0.6,(product11)
3,0.6,(product7)
4,0.6,(product9)
5,0.6,"(product6, product11)"
7,0.6,"(product9, product4)"
8,0.6,"(product6, product7)"
9,0.6,"(product9, product6)"


Here we can see the itemsets with minimum support of 60% with the column indices which can be used for some downstream operations such as making marketing strategies like giving some offers in buying combinations of products.

Now let’s have a look at FP Growth Algorithm.

### FP-Growth
---
The FP-Growth Algorithm is an alternative way to find frequent item sets without using candidate generations, thus improving performance. As you can see from Apriori algorithm, it generate candidate to create itemsets by scanning whole database go back and forth. That's why FP-Growth faster. The core of this method is the usage of a special data structure named frequent-pattern tree (FP-tree), which retains the item set association information.

#### FP-Tree
![](https://static.javatpoint.com/tutorial/data-mining/images/fp-growth-algorithm-in-data-mining.png)

The frequent-pattern tree (FP-tree) is a compact data structure that stores quantitative information about frequent patterns in a database. Each transaction is read and then mapped onto a path in the FP-tree. This is done until all transactions have been read. Different transactions with common subsets allow the tree to remain compact because their paths overlap.

A frequent Pattern Tree is made with the initial item sets of the database. The purpose of the FP tree is to mine the most frequent pattern. Each node of the FP tree represents an item of the item set.

The root node represents null, while the lower nodes represent the item sets. The associations of the nodes with the lower nodes, that is, the item sets with the other item sets, are maintained while forming the tree.

**FP-tree structure given below:**

1. One root is labelled as "null" with a set of item-prefix subtrees as children and a frequent-item-header table.
2. Each node in the item-prefix subtree consists of three fields:
    - Item-name: registers which item is represented by the node;
    - Count: the number of transactions represented by the portion of the path reaching the node;
    - Node-link: links to the next node in the FP-tree carrying the same item name or null if there is none.
3. Each entry in the frequent-item-header table consists of two fields:
    - Item-name: as the same to the node;
    - Head of node-link: a pointer to the first node in the FP-tree carrying the item name.
    
Additionally, the frequent-item-header table can have the count support for an item. The below diagram is an example of a best-case scenario that occurs when all transactions have the same itemset; the size of the FP-tree will be only a single branch of nodes.

**The FP-tree is constructed as follows:**

1. The first step is to scan the database to find the occurrences of the itemsets in the database. This step is the same as the first step of Apriori. The count of 1-itemsets in the database is called support count or frequency of 1-itemset.
2. The second step is to construct the FP tree. For this, create the root of the tree. The root is represented by null.
3. The next step is to scan the database again and examine the transactions. Examine the first transaction and find out the itemset in it. The itemset with the max count is taken at the top, and then the next itemset with the lower count. It means that the branch of the tree is constructed with transaction itemsets in descending order of count.
4. The next transaction in the database is examined. The itemsets are ordered in descending order of count. If any itemset of this transaction is already present in another branch, then this transaction branch would share a common prefix to the root.
This means that the common itemset is linked to the new node of another itemset in this transaction.
5. Also, the count of the itemset is incremented as it occurs in the transactions. The common node and new node count are increased by 1 as they are created and linked according to transactions.
6. The next step is to mine the created FP Tree. For this, the lowest node is examined first, along with the links of the lowest nodes. The lowest node represents the frequency pattern length 1. From this, traverse the path in the FP Tree. This path or paths is called a conditional pattern base.
A conditional pattern base is a sub-database consisting of prefix paths in the FP tree occurring with the lowest node (suffix).
7. Construct a Conditional FP Tree, formed by a count of itemsets in the path. The itemsets meeting the threshold support are considered in the Conditional FP Tree.
8. Frequent Patterns are generated from the Conditional FP Tree.

Using this algorithm, the FP-tree is constructed in two database scans. The first scan collects and sorts the set of frequent items, and the second constructs the FP-Tree.

**Example:**

Support threshold=50%, Confidence= 60%

**Table1:**  
| Transaction | List of items |
|-------------|---------------|
| T1          | I1,I2,I3      |
| T2          | I2,I3,I4      |
| T3          | I4,I5         |
| T4          | I1,I2,I4      |
| T5          | I1,I2,I3,I5   |
| T6          | I1,I2,I3,I4   |

Solution: Support threshold=50% => 0.5*6= 3 => min_sup=3

**Table 2: Count of each item**  
| Item | Count |
|------|-------|
| I1   | 4     |
| I2   | 5     |
| I3   | 4     |
| I4   | 4     |
| I5   | 2     |

**Table 3: Sort the itemset in descending order.**  
| Item | Count |
|------|-------|
| I2   | 5     |
| I1   | 4     |
| I3   | 4     |
| I4   | 4     |

Let's build the FP tree in the following steps, such as:

1. Considering the root node null.
2. The first scan of Transaction T1: I1, I2, I3 contains three items {I1:1}, {I2:1}, {I3:1}, where I2 is linked as a child, I1 is linked to I2 and I3 is linked to I1.
3. T2: I2, I3, and I4 contain I2, I3, and I4, where I2 is linked to root, I3 is linked to I2 and I4 is linked to I3. But this branch would share the I2 node as common as it is already used in T1.
4. Increment the count of I2 by 1, and I3 is linked as a child to I2, and I4 is linked as a child to I3. The count is {I2:2}, {I3:1}, {I4:1}.
5. T3: I4, I5. Similarly, a new branch with I5 is linked to I4 as a child is created.
6. T4: I1, I2, I4. The sequence will be I2, I1, and I4. I2 is already linked to the root node. Hence it will be incremented by 1. Similarly I1 will be incremented by 1 as it is already linked with I2 in T1, thus {I2:3}, {I1:2}, {I4:1}.
7. T5:I1, I2, I3, I5. The sequence will be I2, I1, I3, and I5. Thus {I2:4}, {I1:3}, {I3:2}, {I5:1}.
8. T6: I1, I2, I3, I4. The sequence will be I2, I1, I3, and I4. Thus {I2:5}, {I1:4}, {I3:3}, {I4 1}.

![](https://static.javatpoint.com/tutorial/data-mining/images/fp-growth-algorithm-in-data-mining3.png)

Mining of FP-tree is summarized below:

1. The lowest node item, I5, is not considered as it does not have a min support count. Hence it is deleted.
2. The next lower node is I4. I4 occurs in 2 branches , {I2,I1,I3,I4:1},{I2,I3,I4:1}. Therefore considering I4 as suffix the prefix paths will be {I2, I1, I3:1}, {I2, I3: 1} this forms the conditional pattern base.
3. The conditional pattern base is considered a transaction database, and an FP tree is constructed. This will contain {I2:2, I3:2}, I1 is not considered as it does not meet the min support count.
4. This path will generate all combinations of frequent patterns : {I2,I4:2},{I3,I4:2},{I2,I3,I4:2}
5. For I3, the prefix path would be: {I2,I1:3},{I2:1}, this will generate a 2 node FP-tree : {I2:4, I1:3} and frequent patterns are generated: {I2,I3:4}, {I1:I3:3}, {I2,I1,I3:3}.
6. For I1, the prefix path would be: {I2:4} this will generate a single node FP-tree: {I2:4} and frequent patterns are generated: {I2, I1:4}.

| Item | Conditional Pattern Base | Conditional FP-tree | Frequent Patterns Generated        |
|------|--------------------------|---------------------|------------------------------------|
| I4   | {I2,I1,I3:1},{I2,I3:1}   | {I2:2, I3:2}        | {I2,I4:2},{I3,I4:2},{I2,I3,I4:2}   |
| I3   | {I2,I1:3},{I2:1}         | {I2:4, I1:3}        | {I2,I3:4}, {I1:I3:3}, {I2,I1,I3:3} |
| I1   | {I2:4}                   | {I2:4}              | {I2,I1:4}                          |

The diagram given below depicts the conditional FP tree associated with the conditional node I3.

![](https://static.javatpoint.com/tutorial/data-mining/images/fp-growth-algorithm-in-data-mining4.png)

In [ ]:
new_dataset = []

for v in dataset:
    new_dataset.append(["|".join(v)])

new_dataset

Out[6]: [['product7|product9|product8|product6|product4|product11'],
 ['product3|product9|product8|product6|product4|product11'],
 ['product7|product1|product6|product4'],
 ['product7|product10|product2|product6|product11'],
 ['product2|product9|product6|product5|product4']]

In [ ]:
schema = StructType([StructField("items", StringType(), True)])

spark_df = spark.createDataFrame(data=new_dataset, schema=schema)
spark_df = spark_df.withColumn('items', split(col('items'), r'\|'))
spark_df.show(truncate=False)

+-------------------------------------------------------------+
|items                                                        |
+-------------------------------------------------------------+
|[product7, product9, product8, product6, product4, product11]|
|[product3, product9, product8, product6, product4, product11]|
|[product7, product1, product6, product4]                     |
|[product7, product10, product2, product6, product11]         |
|[product2, product9, product6, product5, product4]           |
+-------------------------------------------------------------+



In [ ]:
fp = FPGrowth(minSupport=0.6)
fpm = fp.fit(spark_df)
fpm.freqItemsets.sort("freq", ascending=False).withColumn('support', col('freq')/5).show(truncate=False)

+------------------------------+----+-------+
|items                         |freq|support|
+------------------------------+----+-------+
|[product6]                    |5   |1.0    |
|[product4]                    |4   |0.8    |
|[product4, product6]          |4   |0.8    |
|[product11]                   |3   |0.6    |
|[product7]                    |3   |0.6    |
|[product11, product6]         |3   |0.6    |
|[product9]                    |3   |0.6    |
|[product7, product6]          |3   |0.6    |
|[product9, product4]          |3   |0.6    |
|[product9, product4, product6]|3   |0.6    |
|[product9, product6]          |3   |0.6    |
+------------------------------+----+-------+



### Difference between Apriori and FP Growth
| Apriori                                                                                                                                | FP Growth                                                                                 |
|----------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| Apriori generates frequent patterns by making the itemsets using pairings such as single item set, double itemset, and triple itemset. | FP Growth generates an FP-Tree for making frequent patterns.                              |
| Apriori uses candidate generation where frequent subsets are extended one item at a time.                                              | FP-growth generates a conditional FP-Tree for every item in the data.                     |
| Since apriori scans the database in each step, it becomes time-consuming for data where the number of items is larger.                 | FP-tree requires only one database scan in its beginning steps, so it consumes less time. |
| A converted version of the database is saved in the memory                                                                             | A set of conditional FP-tree for every item is saved in the memory                        |
| It uses a breadth-first search                                                                                                         | It uses a depth-first search.                                                             |

## Association Rule
---
![](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1530898580/Image_1_ip8nzc.png)

Association rule mining is a technique used to identify patterns in large data sets. It involves finding relationships between variables in the data and using those relationships to make predictions or decisions. The goal of association rule mining is to uncover rules that describe the relationships between different items in the data set.

For example, consider a dataset of transactions at a grocery store. Association rule mining could be used to identify relationships between items that are frequently purchased together. For example, the rule "If a customer buys bread, they are also likely to buy milk" is an association rule that could be mined from this data set. We can use such rules to inform decisions about store layout, product placement, and marketing efforts.

#### Support
Support is a measure of how frequently an item or itemset appears in the dataset. It is calculated as the number of transactions containing the item(s) divided by the total number of transactions in the dataset. High support indicates that an item or itemset is common in the dataset, while low support indicates that it is rare.

![](https://res.cloudinary.com/dyd911kmh/image/upload/v1674491861/Support_Formula_e4d9c995bb.png)

#### Confidence
Confidence is a measure of the strength of the association between two items. It is calculated as the number of transactions containing both items divided by the number of transactions containing the first item. High confidence indicates that the presence of the first item is a strong predictor of the presence of the second item.

![](https://res.cloudinary.com/dyd911kmh/image/upload/v1674491862/Confidence_Formula_eade731502.png)

![](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1551286006/APRIORI3_r4im8b.jpg)

#### Lift
Lift is a measure of the strength of the association between two items, taking into account the frequency of both items in the dataset. It is calculated as the confidence of the association divided by the support of the second item. Lift is used to compare the strength of the association between two items to the expected strength of the association if the items were independent. 

A lift value greater than 1 indicates that the association between two items is stronger than expected based on the frequency of the individual items. This suggests that the association may be meaningful and worth further investigation. A lift value less than 1 indicates that the association is weaker than expected and may be less reliable or less significant.

![](https://res.cloudinary.com/dyd911kmh/image/upload/v1674491862/Lift_Formula_0c1b28e4d5.png)

In [ ]:
fpm.associationRules.sort("antecedent", "consequent").show()

+--------------------+----------+----------+----+-------+
|          antecedent|consequent|confidence|lift|support|
+--------------------+----------+----------+----+-------+
|         [product11]|[product6]|       1.0| 1.0|    0.6|
|          [product4]|[product6]|       1.0| 1.0|    0.8|
|          [product6]|[product4]|       0.8| 1.0|    0.8|
|          [product7]|[product6]|       1.0| 1.0|    0.6|
|          [product9]|[product4]|       1.0|1.25|    0.6|
|          [product9]|[product6]|       1.0| 1.0|    0.6|
|[product9, product4]|[product6]|       1.0| 1.0|    0.6|
|[product9, product6]|[product4]|       1.0|1.25|    0.6|
+--------------------+----------+----------+----+-------+



## References
---
- [FP Growth Algorithm in Data Mining](https://www.javatpoint.com/fp-growth-algorithm-in-data-mining#:~:text=What%20is%20FP%20Growth%20Algorithm,divide%2Dand%2Dconquer%20strategy.)
- [Apriori vs FP-Growth in Market Basket Analysis – A Comparative Guide](https://analyticsindiamag.com/apriori-vs-fp-growth-in-market-basket-analysis-a-comparative-guide/#:~:text=Apriori%20uses%20candidate%20generation%20where,number%20of%20items%20is%20larger.)
- [Market Basket Analysis using R](https://www.datacamp.com/tutorial/market-basket-analysis-r)
- [Market Basket Analysis: A Comprehensive Guide for Businesses](https://www.analyticsvidhya.com/blog/2021/10/a-comprehensive-guide-on-market-basket-analysis/)